In [2]:
import os
import pandas as pd
import pickle
import biotype.similarity as sim

In [3]:
DATA_DIR_PATH = "datasets/export"
META_DIR_PATH = "doc"
PEAK_DIR_PATH = "extracted_peaks"
files =  os.listdir(DATA_DIR_PATH)
files = [fn  for fn in files if fn.endswith(".txt")]
files = sorted(files)
for file in files[:5] :
    print(file)

L100_0_G7_1.txt
L100_0_G8_1.txt
L101_0_A1_1.txt
L101_0_A2_1.txt
L102_0_A3_1.txt


In [4]:
meta = pd.read_csv(META_DIR_PATH + '/meta.csv', index_col=0,encoding='utf-8')
meta[['菌株名','血清型']].head()

,菌株名,血清型
MALDITOFMS Listeria serial No.,,
L001,LM1,1/2a
L002,LM3,1/2a
L003,LM4,4b
L004,LM7,1/2a
L005,LM8,1/2a


In [5]:
serotype = meta['血清型'].to_dict()

In [6]:
peaks_extracted = []
n = len(files)
for i in range(n):
    df = None
    df = pd.read_table(f"{DATA_DIR_PATH}/{files[i]}",sep=" ", header=None,names=['m/z', 'intensity']) 
    x, y = df['m/z'].to_numpy(), df['intensity'].to_numpy()

    pickle_file = files[i][:-4] # trim '.txt'
    with open(f'{PEAK_DIR_PATH}/{pickle_file}_peaks.pkl', 'rb') as peak_file:
        peaks = pickle.load(peak_file)
        st = serotype.get(files[i][:4],'NA')
        print(f'{pickle_file} ({st}) peaks: {len(peaks)}')
        peaks_extracted +=  [(x[peaks], y[peaks]) ] 
        

L100_0_G7_1 (1/2b) peaks: 208
L100_0_G8_1 (1/2b) peaks: 226
L101_0_A1_1 (1/2a) peaks: 226
L101_0_A2_1 (1/2a) peaks: 195
L102_0_A3_1 (1/2b) peaks: 208
L102_0_A4_1 (1/2b) peaks: 200
L103_0_A5_1 (1/2b) peaks: 192
L103_0_A6_1 (1/2b) peaks: 188
L104_0_A7_1 (1/2a) peaks: 193
L104_0_A8_1 (1/2a) peaks: 221
L105_0_A10_1 (1/2b) peaks: 186
L105_0_A9_1 (1/2b) peaks: 204
L106_0_A11_1 (1/2a) peaks: 208
L106_0_A12_1 (1/2a) peaks: 196
L107_0_B1_1 (1/2b) peaks: 190
L107_0_B2_1 (1/2b) peaks: 195
L108_0_B3_1 (UT) peaks: 213
L108_0_B4_1 (UT) peaks: 186
L109_0_B5_1 (UT) peaks: 214
L109_0_B6_1 (UT) peaks: 205
L10_0_G10_1 (NA) peaks: 189
L10_0_G9_1 (NA) peaks: 197
L110_0_B7_1 (UT) peaks: 190
L110_0_B8_1 (UT) peaks: 203
L111_0_B10_1 (UT) peaks: 240
L111_0_B9_1 (UT) peaks: 214
L112_0_B11_1 (1/2a) peaks: 226
L112_0_B12_1 (1/2a) peaks: 223
L113_0_C1_1 (UT) peaks: 202
L113_0_C2_1 (UT) peaks: 210
L114_0_C3_1 (UT) peaks: 289
L114_0_C4_1 (UT) peaks: 265
L115_0_C5_1 (3a) peaks: 207
L115_0_C6_1 (3a) peaks: 195
L116_0_

In [7]:
import importlib
n = len(files)

In [8]:
importlib.reload(sim)
rs = []
print('Jaccard_Similarity:')
for i in range(n-1):
    for j in range(i+1,n):
        score = sim.similar_to(peaks_extracted[i], peaks_extracted[j], method='jaccard', rank=2)
        rs += [(i, j, score)]

rs = sorted(rs, key=lambda x: x[2], reverse=True)
algo_score=0
for k, v in enumerate(rs):
    i, j, s = v
    tag = '*' if files[i][:4]==files[j][:4] else '' # tag the same strain id
    st1 =  serotype.get(files[i][:4],'NA')
    st2 =  serotype.get(files[j][:4],'NA')
    if tag=='*':
        algo_score += k
        print(f'#{k+1:2d}{tag}:\t{files[i]}[{st1}] & {files[j]}[{st2}]\t{s:.4f}{tag}')

p = n/2      # pairs of files   
print(f'algo_score={2*algo_score/float(p*(p+1)):.2f} (closer to 1.0 is better )')

Jaccard_Similarity:
# 6*:	L121_0_D5_1.txt[1/2b] & L121_0_D6_1.txt[1/2b]	0.4802*
# 7*:	L129_0_E10_1.txt[1/2b] & L129_0_E9_1.txt[1/2b]	0.4781*
#12*:	L128_0_E7_1.txt[1/2a] & L128_0_E8_1.txt[1/2a]	0.4715*
#16*:	L12_0_H1_1.txt[NA] & L12_0_H2_1.txt[NA]	0.4683*
#21*:	L116_0_C7_1.txt[nan] & L116_0_C8_1.txt[nan]	0.4672*
#23*:	L138_0_G3_1.txt[1/2c] & L138_0_G4_1.txt[1/2c]	0.4662*
#43*:	L126_0_E3_1.txt[1/2b] & L126_0_E4_1.txt[1/2b]	0.4474*
#45*:	L122_0_D7_1.txt[1/2b] & L122_0_D8_1.txt[1/2b]	0.4472*
#49*:	L123_0_D10_1.txt[1/2a] & L123_0_D9_1.txt[1/2a]	0.4457*
#51*:	L119_0_D1_1.txt[1/2a] & L119_0_D2_1.txt[1/2a]	0.4440*
#60*:	L139_0_G5_1.txt[1/2c] & L139_0_G6_1.txt[1/2c]	0.4424*
#65*:	L13_0_H3_1.txt[NA] & L13_0_H4_1.txt[NA]	0.4407*
#69*:	L110_0_B7_1.txt[UT] & L110_0_B8_1.txt[UT]	0.4396*
#80*:	L107_0_B1_1.txt[1/2b] & L107_0_B2_1.txt[1/2b]	0.4366*
#89*:	L132_0_F3_1.txt[4b] & L132_0_F4_1.txt[4b]	0.4343*
#108*:	L101_0_A1_1.txt[1/2a] & L101_0_A2_1.txt[1/2a]	0.4320*
#110*:	L102_0_A3_1.txt[1/2b] & L102_0_A

In [9]:
importlib.reload(sim)
rs = []
print('Rank_Similarity:')
for i in range(n-1):
    for j in range(i+1,n):
        score = sim.similar_to(peaks_extracted[i], peaks_extracted[j], method='rank', rank=2)
        rs += [(i, j, score)]

rs = sorted(rs, key=lambda x: x[2], reverse=True)
algo_score=0
for k, v in enumerate(rs):
    i, j, s = v
    tag = '*' if files[i][:4]==files[j][:4] else '' # tag the same strain id
    st1 =  serotype.get(files[i][:4],'NA')
    st2 =  serotype.get(files[j][:4],'NA')
    if tag=='*':
        algo_score += k
        print(f'#{k+1:2d}{tag}:\t{files[i]}[{st1}] & {files[j]}[{st2}]\t{s:.4f}{tag}')

p = n/2      # pairs of files   
print(f'algo_score={2*algo_score/float(p*(p+1)):.2f} (closer to 1.0 is better )')

Rank_Similarity:
# 1*:	L138_0_G3_1.txt[1/2c] & L138_0_G4_1.txt[1/2c]	0.1917*
# 2*:	L115_0_C5_1.txt[3a] & L115_0_C6_1.txt[3a]	0.1731*
# 4*:	L12_0_H1_1.txt[NA] & L12_0_H2_1.txt[NA]	0.1548*
# 7*:	L116_0_C7_1.txt[nan] & L116_0_C8_1.txt[nan]	0.1423*
#10*:	L122_0_D7_1.txt[1/2b] & L122_0_D8_1.txt[1/2b]	0.1408*
#13*:	L121_0_D5_1.txt[1/2b] & L121_0_D6_1.txt[1/2b]	0.1349*
#19*:	L129_0_E10_1.txt[1/2b] & L129_0_E9_1.txt[1/2b]	0.1275*
#28*:	L120_0_D3_1.txt[1/2a] & L120_0_D4_1.txt[1/2a]	0.1209*
#32*:	L119_0_D1_1.txt[1/2a] & L119_0_D2_1.txt[1/2a]	0.1191*
#43*:	L13_0_H3_1.txt[NA] & L13_0_H4_1.txt[NA]	0.1148*
#61*:	L102_0_A3_1.txt[1/2b] & L102_0_A4_1.txt[1/2b]	0.1088*
#65*:	L128_0_E7_1.txt[1/2a] & L128_0_E8_1.txt[1/2a]	0.1065*
#99*:	L113_0_C1_1.txt[UT] & L113_0_C2_1.txt[UT]	0.0990*
#102*:	L136_0_F11_1.txt[4b] & L136_0_F12_1.txt[4b]	0.0988*
#108*:	L123_0_D10_1.txt[1/2a] & L123_0_D9_1.txt[1/2a]	0.0978*
#109*:	L131_0_F1_1.txt[1/2a] & L131_0_F2_1.txt[1/2a]	0.0974*
#177*:	L118_0_C11_1.txt[1/2a] & L118_0_C12

In [10]:
importlib.reload(sim)
rs = []
print('Weighted_Similarity:')
for i in range(n-1):
    for j in range(i+1,n):
        score = sim.similar_to(peaks_extracted[i], peaks_extracted[j], method='weighted', rank=2)
        rs += [(i, j, score)]

rs = sorted(rs, key=lambda x: x[2], reverse=True) # sort all results in decending of similarity scores
algo_score = 0
for k, v in enumerate(rs):
    i, j, s = v
    tag = '*' if files[i][:4]==files[j][:4] else '' # tag the same strain id
    if tag=='*':
        print(f'#{k+1:2d}{tag}:\t{files[i]} & {files[j]}\t{s:.4f}{tag}')
        algo_score += k;      

p = n/2      # pairs of files         
print(f'algo_score={2*algo_score/float(p*(p+1)):.2f} (closer to 1.0 is better )')

Weighted_Similarity:
# 1*:	L138_0_G3_1.txt & L138_0_G4_1.txt	0.0486*
# 2*:	L115_0_C5_1.txt & L115_0_C6_1.txt	0.0443*
# 4*:	L12_0_H1_1.txt & L12_0_H2_1.txt	0.0424*
# 8*:	L121_0_D5_1.txt & L121_0_D6_1.txt	0.0406*
#10*:	L129_0_E10_1.txt & L129_0_E9_1.txt	0.0396*
#18*:	L128_0_E7_1.txt & L128_0_E8_1.txt	0.0380*
#20*:	L116_0_C7_1.txt & L116_0_C8_1.txt	0.0378*
#22*:	L13_0_H3_1.txt & L13_0_H4_1.txt	0.0376*
#27*:	L119_0_D1_1.txt & L119_0_D2_1.txt	0.0371*
#28*:	L122_0_D7_1.txt & L122_0_D8_1.txt	0.0369*
#49*:	L102_0_A3_1.txt & L102_0_A4_1.txt	0.0353*
#56*:	L120_0_D3_1.txt & L120_0_D4_1.txt	0.0348*
#71*:	L123_0_D10_1.txt & L123_0_D9_1.txt	0.0342*
#92*:	L118_0_C11_1.txt & L118_0_C12_1.txt	0.0334*
#108*:	L113_0_C1_1.txt & L113_0_C2_1.txt	0.0329*
#161*:	L110_0_B7_1.txt & L110_0_B8_1.txt	0.0317*
#163*:	L131_0_F1_1.txt & L131_0_F2_1.txt	0.0317*
#171*:	L136_0_F11_1.txt & L136_0_F12_1.txt	0.0316*
#181*:	L139_0_G5_1.txt & L139_0_G6_1.txt	0.0312*
#223*:	L105_0_A10_1.txt & L105_0_A9_1.txt	0.0304*
#236*:	L10